# explorer

> CLI tool for API exploration of cjm-fasthtml-tailwind utilities

In [ ]:
#| default_exp cli.explorer

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Tuple, Any, Optional
from cjm_fasthtml_tailwind.cli.utils import (
    SearchResult, discover_utility_modules,
    load_code_from_file, display_items_generic, handle_module_not_found,
    find_usage_in_items, get_view_command, format_usage_examples, simple_item_formatter,
    indented_item_formatter, list_utility_modules
)
from cjm_fasthtml_tailwind.cli.factory_extraction import (
    list_all_factories, list_module_factories, get_factory_by_name
)
from cjm_fasthtml_tailwind.cli.example_discovery import (
    list_all_examples, get_example_by_name, list_module_examples
)
from cjm_fasthtml_tailwind.cli.helper_discovery import get_module_helpers
from cjm_fasthtml_tailwind.cli.search import search_all
from cjm_fasthtml_tailwind.cli.core_utils_discovery import get_core_utilities
from cjm_fasthtml_tailwind.cli.test_code import execute_test_code
from cjm_fasthtml_tailwind.cli.imports import get_recommended_imports
from cjm_fasthtml_tailwind.cli.pattern_scanner import (
    analyze_and_suggest_input, display_input_analysis, 
    InputType, detect_input_type
)

## Display Functions

Functions to format and display the discovered information:

In [ ]:
#| export
def display_modules(
): # TODO: Add type hint
    """Display all available utility modules with their documentation."""
    modules = list_utility_modules()
    
    # Convert dict to list of tuples for the formatter
    module_list = [(name, doc) for name, doc in modules.items()]
    
    # Create formatter
    def module_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        name, doc = item
        return f"{name}: {doc}"
    
    # Instructions
    instructions = []
    if modules:
        first_module = list(modules.keys())[0]
        instructions.extend([
            ("To explore a specific module:", "cjm-tailwind-explore factories -m <module>"),
            ("Example:", f"cjm-tailwind-explore factories -m {first_module}")
        ])
    
    display_items_generic(
        items=module_list,
        title="Available Utility Modules:",
        item_formatter=module_formatter,
        item_type="modules",
        instructions=instructions
    )

In [ ]:
#| export
def display_module_factories(
    module_name: str  # TODO: Add description
): # TODO: Add type hint
    """Display all factories in a specific module."""
    factories = list_module_factories(module_name)
    
    if not factories:
        handle_module_not_found("factories", module_name)
        return
    
    print(f"Factories in '{module_name}' module:")
    print("=" * 60, "\n")

    for factory_info in factories:
        print(f"{factory_info.name}: {factory_info.doc}")
    
    print(f"\nTotal factories: {len(factories)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View detailed factory info:")
    if factories:
        first_factory = factories[0].name
        print(f"    cjm-tailwind-explore factory {module_name} {first_factory}")
    
    print(f"\n  View usage examples:")
    print(f"    cjm-tailwind-explore examples -m {module_name}")
    
    print(f"\n  View helper functions:")
    print(f"    cjm-tailwind-explore helpers -m {module_name}")
    
    print(f"\n  Search within this module:")
    print(f"    cjm-tailwind-explore search <query> --in factories")

In [ ]:
#| export
def display_all_factories(
): # TODO: Add type hint
    """Display all factories across all modules."""
    all_factories = list_all_factories()
    
    # Create formatter using simple_item_formatter
    factory_formatter = simple_item_formatter('name', 'doc')
    indented_formatter = indented_item_formatter("  ")
    
    # Instructions
    instructions = []
    if all_factories:
        first_module = list(all_factories.keys())[0]
        first_factory = all_factories[first_module][0].name if all_factories[first_module] else None
        
        instructions.extend([
            ("View factories for a specific module:", f"cjm-tailwind-explore factories -m {first_module}" if first_module else None),
            ("View detailed factory info:", f"cjm-tailwind-explore factory {first_module} {first_factory}" if first_module and first_factory else None),
            ("Search for specific factories:", "cjm-tailwind-explore search <query> --in factories")
        ])
    
    display_items_generic(
        items=all_factories,
        title="All Factories by Module:",
        item_formatter=indented_formatter(lambda item: f"{item.name} - {item.doc}"),
        item_type="factories",
        instructions=instructions
    )

In [ ]:
#| export
def display_module_examples(
    module_name: str  # TODO: Add description
): # TODO: Add type hint
    """Display all usage examples in a specific module."""
    examples = list_module_examples(module_name)
    
    if not examples:
        handle_module_not_found("examples", module_name)
        return
    
    print(f"Usage Examples in '{module_name}' module:")
    print("=" * 60)
    
    for example_info in examples:
        print(f"\n{example_info.feature.replace('_', ' ').title()} Examples:")
        print(f"Function: {example_info.name}")
        print("-" * 60)
        print(example_info.source)
    
    print(f"\nTotal example functions: {len(examples)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View a specific example:")
    if examples:
        first_example = examples[0]
        print(f"    cjm-tailwind-explore example {module_name} {first_example.feature}")
    
    print(f"\n  View factories in this module:")
    print(f"    cjm-tailwind-explore factories -m {module_name}")
    
    print(f"\n  Search for specific patterns in examples:")
    print(f"    cjm-tailwind-explore search <query> --in examples --include-source")

In [ ]:
#| export
def display_all_examples(
): # TODO: Add type hint
    """Display all usage examples across all modules."""
    all_examples = list_all_examples()
    
    # Create formatter
    def example_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        feature_name = item.feature.replace('_', ' ')
        return f"  - {feature_name}: {item.docstring}"
    
    display_items_generic(
        items=all_examples,
        title="All Usage Examples by Module:",
        item_formatter=example_formatter,
        item_type="example functions"
    )

In [ ]:
#| export
def display_example_source(
    module_name: str,  # TODO: Add description
    feature: str  # TODO: Add description
): # TODO: Add type hint
    """Display the source code of a specific example function."""
    example = get_example_by_name(module_name, feature)
    
    if not example:
        print(f"No example found for feature '{feature}' in module '{module_name}'.")
        print(f"\nTry running 'cjm-tailwind-explore examples --module {module_name}' to see available features.")
        return
    
    print(f"Example: {example.feature.replace('_', ' ').title()} ({example.module_name} module)")
    print(f"Function: {example.name}")
    print("=" * 60)
    print(example.source)

In [ ]:
#| export
def display_module_helpers(
    module_name: str  # TODO: Add description
): # TODO: Add type hint
    """Display helper functions available in a specific module."""
    helpers = get_module_helpers(module_name)
    
    if not helpers:
        handle_module_not_found("helper functions", module_name)
        return
    
    print(f"Helper Functions in '{module_name}' module:")
    print("=" * 60)
    
    for helper_info in helpers:
        print(f"\n{helper_info.name}{helper_info.signature}")
        print("-" * 40)
        print(helper_info.docstring)
    
    print(f"\nTotal helper functions: {len(helpers)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    print(f"  View helper source code:")
    if helpers:
        first_helper = helpers[0].name
        print(f"    cjm-tailwind-explore helper {module_name} {first_helper}")
    
    print(f"\n  View factories in this module:")
    print(f"    cjm-tailwind-explore factories -m {module_name}")
    
    print(f"\n  View usage examples:")
    print(f"    cjm-tailwind-explore examples -m {module_name}")
    
    print(f"\n  Search for helper usage:")
    if helpers:
        print(f"    cjm-tailwind-explore search {helpers[0].name} --include-source")

In [ ]:
#| export
def display_helper_source(
    module_name: str,  # TODO: Add description
    helper_name: str  # TODO: Add description
): # TODO: Add type hint
    """Display the source code of a specific helper function."""
    helpers = get_module_helpers(module_name)
    
    helper_info = None
    for h in helpers:
        if h.name == helper_name:
            helper_info = h
            break
    
    if not helper_info:
        print(f"No helper function '{helper_name}' found in module '{module_name}'.")
        print(f"\nTry running 'cjm-tailwind-explore helpers --module {module_name}' to see available helpers.")
        return
    
    print(f"Helper: {helper_info.name} ({helper_info.module_name} module)")
    print(f"Signature: {helper_info.name}{helper_info.signature}")
    print("=" * 60)
    print(helper_info.source)
    
    # Find test examples that demonstrate this helper
    examples = list_module_examples(module_name)
    if examples:
        # Look for the helper examples test function
        helper_example = None
        for example in examples:
            if example.feature == 'helper' or example.feature == 'helpers':
                helper_example = example
                break
        
        if helper_example:
            # Check if this specific helper is used in the test
            import re
            patterns = [
                rf'\b{helper_name}\(',           # helper_name(
                rf'\bstr\({helper_name}\b',      # str(helper_name
                rf'\({helper_name}\)',           # (helper_name)
            ]
            
            if any(re.search(pattern, helper_example.source) for pattern in patterns):
                print(f"\nTest Example:")
                print(f"  {helper_example.name} ({helper_example.docstring})")
                print(f"  View with: cjm-tailwind-explore example {module_name} {helper_example.feature}")
            else:
                print(f"\nNo test example found that demonstrates the '{helper_name}' helper.")
        else:
            print(f"\nNo helper examples test function found in module '{module_name}'.")

In [ ]:
#| export
def display_all_helpers(
): # TODO: Add type hint
    """Display all helper functions across all modules."""
    all_helpers = {}
    
    for module_name, module in discover_utility_modules():
        helpers = get_module_helpers(module_name)
        if helpers:
            all_helpers[module_name] = helpers
    
    # Create formatter
    def helper_formatter(
        item  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        # Get first line of docstring for concise display
        first_line = item.docstring.split('\n')[0] if item.docstring else "No documentation available"
        return f"  - {item.name}: {first_line}"
    
    display_items_generic(
        items=all_helpers,
        title="All Helper Functions by Module:",
        item_formatter=helper_formatter,
        item_type="helper functions"
    )

In [ ]:
#| export
def display_factory_info(
    module_name: str,  # TODO: Add description
    factory_name: str  # TODO: Add description
): # TODO: Add type hint
    """Display detailed information about a specific factory."""
    factory_info = get_factory_by_name(module_name, factory_name)
    
    if not factory_info:
        print(f"No factory '{factory_name}' found in module '{module_name}'.")
        print(f"\nTry running 'cjm-tailwind-explore factories --module {module_name}' to see available factories.")
        return
    
    print(f"Factory: {factory_info.name} ({factory_info.module_name} module)")
    print("=" * 60)
    
    # Get detailed info from the factory
    try:
        info = factory_info.factory.get_info()
        
        # Display description
        print(f"\nDescription:")
        print(f"  {info['description']}")
        
        # Display valid inputs
        print(f"\nValid Inputs:")
        if isinstance(info['valid_inputs'], list):
            for input_desc in info['valid_inputs']:
                print(f"  - {input_desc}")
        else:
            print(f"  {info['valid_inputs']}")
        
        # Display options
        if 'options' in info:
            print(f"\nOptions:")
            if isinstance(info['options'], dict):
                for key, value in info['options'].items():
                    if isinstance(value, dict):
                        print(f"  {key}:")
                        for sub_key, sub_value in value.items():
                            print(f"    - {sub_key}: {sub_value}")
                    else:
                        print(f"  - {key}: {value}")
            else:
                print(f"  {info['options']}")
            
    except AttributeError:
        print("\nDetailed information not available for this factory.")
        print("Factory type:", type(factory_info.factory).__name__)
    
    # Find test examples that use this factory
    all_examples = list_all_examples()
    example_usage = find_usage_in_items(
        target_name=factory_name,
        items=all_examples,
        source_getter=lambda e: e.source,
        item_type="examples"
    )
    
    if example_usage:
        print(f"\nTest Examples Using This Factory:")
        formatted = format_usage_examples(
            usage_items=example_usage,
            item_name_getter=lambda e: e.name,
            item_type="examples",
            view_command_type="example"
        )
        for line in formatted:
            print(line)
    else:
        print(f"\nNo test examples found that specifically use the '{factory_name}' factory.")
    
    # Find helper functions that use this factory
    all_helpers = {}
    for mod_name, module in discover_utility_modules():
        helpers = get_module_helpers(mod_name)
        if helpers:
            all_helpers[mod_name] = helpers
    
    helper_usage = find_usage_in_items(
        target_name=factory_name,
        items=all_helpers,
        source_getter=lambda h: h.source,
        item_type="helpers"
    )
    
    if helper_usage:
        print(f"\nHelper Functions Using This Factory:")
        for mod_name, helper in helper_usage:
            # Get first line of docstring for concise display
            first_line = helper.docstring.split('\n')[0] if helper.docstring else "No description available"
            print(f"  - {helper.name}: {first_line}")
            print(f"    View with: {get_view_command('helper', mod_name, helper.name)}")
    else:
        print(f"\nNo helper functions found that specifically use the '{factory_name}' factory.")

In [ ]:
#| export
def display_search_results(
    results: List[SearchResult],  # TODO: Add description
    query: str  # TODO: Add description
): # TODO: Add type hint
    """Display search results in a formatted way."""
    if not results:
        print(f"No results found for '{query}'")
        return
    
    print(f"Search Results for '{query}':")
    print("=" * 60)
    
    # Group results by content type
    by_type = {}
    for result in results:
        if result.content_type not in by_type:
            by_type[result.content_type] = []
        by_type[result.content_type].append(result)
    
    # Display results by type
    for content_type, type_results in by_type.items():
        print(f"\n{content_type.title()}s ({len(type_results)} results)")
        print("-" * 40)
        
        for result in type_results:
            print(f"\n  {result.item_name} ({result.module_name} module)")
            print(f"  Found in: {result.match_location}")
            print(f"  Context: {result.match_context}")
            
            # Show how to view the full item
            if result.content_type == 'factory':
                view_cmd = get_view_command('factory', result.module_name, result.item_name)
            elif result.content_type == 'example':
                # Extract feature from name
                feature = result.item_name.replace(f'test_{result.module_name}_', '').replace('_examples', '')
                view_cmd = get_view_command('example', result.module_name, feature)
            elif result.content_type == 'helper':
                view_cmd = get_view_command('helper', result.module_name, result.item_name)
            elif result.content_type == 'module':
                view_cmd = get_view_command('module', result.module_name, result.module_name)
            else:
                view_cmd = ""
            
            if view_cmd:
                print(f"  View with: {view_cmd}")
    
    print(f"\nTotal results: {len(results)}")

In [ ]:
#| export
def display_core_utility_source(
    util_name: str  # TODO: Add description
): # TODO: Add type hint
    """Display the source code of a specific core utility function."""
    utilities = get_core_utilities()
    
    util_info = None
    for u in utilities:
        if u.name == util_name:
            util_info = u
            break
    
    if not util_info:
        print(f"No core utility '{util_name}' found.")
        print(f"\nTry running 'cjm-tailwind-explore core-utils' to see available utilities.")
        return
    
    print(f"Core Utility: {util_info.name}")
    print(f"Module: {util_info.module}")
    print(f"Signature: {util_info.name}{util_info.signature}")
    print("=" * 60)
    print(util_info.source)
    
    # Find examples that use this utility
    print(f"\nFinding usage examples...")
    
    # Search in all test examples
    all_examples = list_all_examples()
    example_usage = find_usage_in_items(
        target_name=util_name,
        items=all_examples,
        source_getter=lambda e: e.source,
        item_type="examples"
    )
    
    if example_usage:
        print(f"\nFound in {len(example_usage)} test examples:")
        formatted = format_usage_examples(
            usage_items=example_usage,
            item_name_getter=lambda e: e.name,
            item_type="examples",
            view_command_type="example"
        )
        for line in formatted:
            print(line)
    
    # Search in helper functions
    all_helpers = {}
    for module_name, module in discover_utility_modules():
        helpers = get_module_helpers(module_name)
        if helpers:
            all_helpers[module_name] = helpers
    
    helper_usage = find_usage_in_items(
        target_name=util_name,
        items=all_helpers,
        source_getter=lambda h: h.source,
        item_type="helpers"
    )
    
    if helper_usage:
        print(f"\nFound in {len(helper_usage)} helper functions:")
        for module_name, helper in helper_usage:
            print(f"  - {helper.name} ({module_name} module)")
            print(f"    View with: {get_view_command('helper', module_name, helper.name)}")
    
    # Suggest search for more examples
    print(f"\nTo find more usage examples:")
    print(f"  cjm-tailwind-explore search {util_name} --include-source")

In [ ]:
#| export
def display_core_utilities(
): # TODO: Add type hint
    """Display all core utility functions."""
    utilities = get_core_utilities()
    
    if not utilities:
        print("No core utilities found.")
        return
    
    print("Core Utility Functions:")
    print("=" * 60)
    
    for util in utilities:
        print(f"\n{util.name}{util.signature}")
        print("-" * 40)
        print(f"Module: {util.module}")
        print(f"Import: {util.import_statement}")
        print(f"\nDocumentation:")
        print(util.docstring)
    
    print(f"\nTotal core utilities: {len(utilities)}")
    
    # Add helpful instructions
    print("\nTo explore further:")
    if utilities:
        first_util = utilities[0]
        print(f"  View source code:")
        print(f"    cjm-tailwind-explore core-util {first_util.name}")
    print(f"\n  Search for usage examples:")
    print(f"    cjm-tailwind-explore search combine_classes --include-source")

In [ ]:
#| export
def display_imports(
    modules: Optional[List[str]] = None  # TODO: Add description
): # TODO: Add type hint
    """Display recommended import statements."""
    imports = get_recommended_imports(modules)
    
    if modules:
        print(f"Recommended imports for modules: {', '.join(modules)}")
    else:
        print("Recommended imports for all modules:")
    print("=" * 60)
    print()
    
    for imp in imports:
        print(imp)
    
    print("\nUsage:")
    print("- Copy these imports to the top of your Python file")
    print("- The imports include all factories, helpers, and core utilities")
    print("- FastHTML components are available via 'from fasthtml.common import *'")
    
    if not modules:
        print("\nTo get imports for specific modules only:")
        print("  cjm-tailwind-explore imports --modules spacing sizing")

In [ ]:
#| export
def display_test_code_result(
    success: bool,  # TODO: Add description
    stdout: str,  # TODO: Add description
    stderr: str,  # TODO: Add description
    code: str  # TODO: Add description
): # TODO: Add type hint
    """Display the results of test code execution."""
    print("Test Code Execution Result:")
    print("=" * 60)
    
    # Show the code that was tested
    print("\nCode tested:")
    print("-" * 40)
    print(code)
    print("-" * 40)
    
    # Show the result
    if success:
        print("\n✅ Code executed successfully!")
        if stdout:
            print("\nOutput:")
            print(stdout)
    else:
        print("\n❌ Code execution failed!")
        if stderr:
            print("\nError:")
            print(stderr)
    
    # Show helpful tips
    if not success:
        print("\nTips:")
        print("- Check that all factory names are correct (use 'cjm-tailwind-explore factories')")
        print("- Verify function signatures (use 'cjm-tailwind-explore helper <module> <name>')")
        print("- Look at examples for usage patterns (use 'cjm-tailwind-explore examples')")
        print("- Search for specific patterns (use 'cjm-tailwind-explore search <query>')")

## Dynamic Example Helpers

Functions to generate dynamic examples for help text:

In [ ]:
#| export
def get_example_modules(
    limit: int = 2  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get example module names dynamically."""
    modules = list_utility_modules()
    if not modules:
        return ""
    
    module_names = list(modules.keys())[:limit]
    return ", ".join(module_names)

In [ ]:
#| export
def get_example_factories(
    module_name: str = None,  # TODO: Add description
    limit: int = 4  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get example factory names dynamically."""
    if module_name:
        factories = list_module_factories(module_name)
    else:
        # Get from first available module
        all_factories = list_all_factories()
        if not all_factories:
            return ""
        
        first_module = list(all_factories.keys())[0]
        factories = all_factories[first_module]
    
    if not factories:
        return ""
    
    factory_names = [f.name for f in factories[:limit]]
    return ", ".join(factory_names)

In [ ]:
#| export
def get_example_features(
    module_name: str = None,  # TODO: Add description
    limit: int = 3  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get example feature names dynamically."""
    if module_name:
        examples = list_module_examples(module_name)
    else:
        # Get from first module with examples
        all_examples = list_all_examples()
        if not all_examples:
            return ""
        
        first_module = list(all_examples.keys())[0]
        examples = all_examples[first_module]
    
    if not examples:
        return ""
    
    feature_names = [e.feature for e in examples[:limit]]
    return ", ".join(feature_names)

In [ ]:
#| export
def get_example_helpers(
    module_name: str = None,  # TODO: Add description
    limit: int = 2  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get example helper function names dynamically."""
    if module_name:
        helpers = get_module_helpers(module_name)
    else:
        # Get from first module with helpers
        for mod_name, module in discover_utility_modules():
            helpers = get_module_helpers(mod_name)
            if helpers:
                break
        else:
            return ""
    
    if not helpers:
        return ""
    
    helper_names = [h.name for h in helpers[:limit]]
    return ", ".join(helper_names)

In [ ]:
#| export
def get_example_core_utils(
    limit: int = 2  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get example core utility names dynamically."""
    utilities = get_core_utilities()
    if not utilities:
        return ""
    
    util_names = [u.name for u in utilities[:limit]]
    return ", ".join(util_names)

In [ ]:
#| export
def get_combine_classes_example(
) -> str:  # TODO: Add return description
    """Get a dynamic example of combine_classes usage."""
    # Try to find combine_classes usage in examples
    from cjm_fasthtml_tailwind.cli.example_discovery import list_all_examples
    from cjm_fasthtml_tailwind.cli.search import search_all
    
    # Search for combine_classes usage
    results = search_all('combine_classes', include_source=True, content_types=['examples'])
    
    if results:
        # Look for a simple combine_classes call
        import re
        for result in results:
            # Try to extract a combine_classes call from the match context
            match = re.search(r'combine_classes\([^)]+\)', result.match_context)
            if match:
                example = match.group(0)
                # Make sure it's not too long
                if len(example) <= 50:
                    return f"print({example})"
    
    # If no example found, create a minimal one with discovered factories
    factories = get_example_factories(limit=2).split(', ')
    if len(factories) >= 2:
        return f"print(combine_classes({factories[0]}(4), {factories[1]}.auto))"
    
    return "print(combine_classes(...))"

In [ ]:
#| export
def get_example_test_code(
    limit_chars: int = 50  # TODO: Add description
) -> str:  # TODO: Add return description
    """Get an actual code example from test functions."""
    # Try to get examples from the first available module
    all_examples = list_all_examples()
    if not all_examples:
        return "<code-here>"
    
    # Get the first module with examples
    first_module_name = list(all_examples.keys())[0]
    examples = all_examples[first_module_name]
    
    if not examples:
        return "<code-here>"
    
    # Look for a basic example first
    for example in examples:
        if 'basic' in example.feature:
            # Extract a simple assertion from the source
            lines = example.source.strip().split('\n')
            for line in lines:
                line = line.strip()
                # Look for simple assert statements
                if line.startswith('assert') and 'str(' in line and '==' in line:
                    # Extract the expression being tested
                    parts = line.split('==')
                    if len(parts) == 2:
                        expr = parts[0].replace('assert', '').strip()
                        if len(expr) <= limit_chars:
                            return f"print({expr})"
    
    # If no basic example, try to find any simple assertion
    for example in examples:
        lines = example.source.strip().split('\n')
        for line in lines:
            line = line.strip()
            if line.startswith('assert') and 'str(' in line and '==' in line:
                parts = line.split('==')
                if len(parts) == 2:
                    expr = parts[0].replace('assert', '').strip()
                    if len(expr) <= limit_chars:
                        return f"print({expr})"
    
    # Last resort: try to find any factory usage
    for example in examples:
        import re
        # Look for factory calls like p(4), m.auto, w.full
        matches = re.findall(r'\b([a-z]+(?:\.[a-z_]+)?(?:\([^)]+\))?)\b', example.source)
        for match in matches:
            if '(' in match or '.' in match:
                if len(match) <= 20:  # Keep it short
                    return f"print(str({match}))"
    
    return "<code-here>"

## Parser Setup Functions

Helper functions to set up argument parsers for each command:

In [ ]:
#| export
def add_modules_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'modules' command parser."""
    return subparsers.add_parser('modules', help='List all utility modules')

In [ ]:
#| export
def add_factories_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'factories' command parser."""
    # Get dynamic examples
    example_factories = get_example_factories()
    
    description = (
        'List factory instances that generate Tailwind CSS utility classes.\n\n'
        'Factories are the primary way to create CSS classes in this library.'
    )
    
    if example_factories:
        description += f'\nExamples: {example_factories}'
    
    parser = subparsers.add_parser(
        'factories', 
        help='List factories',
        description=description
    )
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show factories for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_factory_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'factory' command parser."""
    parser = subparsers.add_parser('factory', help='Show detailed info for a specific factory')
    
    # Get examples dynamically
    example_modules = get_example_modules()
    example_factories = get_example_factories()
    
    parser.add_argument(
        'module',
        type=str,
        help=f'Module name (e.g., {example_modules})' if example_modules else 'Module name'
    )
    parser.add_argument(
        'name',
        type=str,
        help=f'Factory name (e.g., {example_factories})' if example_factories else 'Factory name'
    )
    return parser

In [ ]:
#| export
def add_examples_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'examples' command parser."""
    parser = subparsers.add_parser('examples', help='Show usage examples')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show examples for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_example_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'example' command parser."""
    parser = subparsers.add_parser('example', help='Show source code for a specific example')
    
    # Get examples dynamically
    example_modules = get_example_modules()
    example_features = get_example_features()
    
    parser.add_argument(
        'module',
        type=str,
        help=f'Module name (e.g., {example_modules})' if example_modules else 'Module name'
    )
    parser.add_argument(
        'feature',
        type=str,
        help=f'Feature name (e.g., {example_features})' if example_features else 'Feature name'
    )
    return parser

In [ ]:
#| export
def add_helpers_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'helpers' command parser."""
    parser = subparsers.add_parser('helpers', help='Show helper functions')
    parser.add_argument(
        '--module', '-m',
        type=str,
        help='Show helpers for a specific module only'
    )
    return parser

In [ ]:
#| export
def add_helper_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'helper' command parser."""
    parser = subparsers.add_parser('helper', help='Show source code for a specific helper')
    
    # Get examples dynamically
    example_modules = get_example_modules()
    example_helpers = get_example_helpers()
    
    parser.add_argument(
        'module',
        type=str,
        help=f'Module name (e.g., {example_modules})' if example_modules else 'Module name'
    )
    parser.add_argument(
        'name',
        type=str,
        help=f'Helper function name (e.g., {example_helpers})' if example_helpers else 'Helper function name'
    )
    return parser

In [ ]:
#| export
def add_search_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'search' command parser."""
    # Get dynamic examples
    example_factories = get_example_factories(limit=2)
    
    description = (
        'Search across all library components to find specific functionality.\n\n'
        'This command searches through:\n'
        '- Factory names and documentation\n'
        '- Example function names and docstrings\n'
        '- Helper function names and docstrings\n'
        '- Module names and documentation\n'
        '- Optionally: source code of examples and helpers\n\n'
        'Use --include-source to search within the actual code implementations.'
    )
    
    if example_factories:
        description += f'\n\nExample searches: {example_factories}, padding, negative, helper'
    
    parser = subparsers.add_parser(
        'search', 
        help='Search across all library components',
        description=description
    )
    parser.add_argument(
        'query',
        type=str,
        help='Search query'
    )
    parser.add_argument(
        '--in',
        dest='search_in',
        type=str,
        choices=['all', 'factories', 'examples', 'helpers', 'modules'],
        default='all',
        help='What to search in (default: all)'
    )
    parser.add_argument(
        '--include-source',
        action='store_true',
        help='Also search in source code of examples and helpers'
    )
    parser.add_argument(
        '--case-sensitive',
        action='store_true',
        help='Perform case-sensitive search'
    )
    return parser

In [ ]:
#| export
def add_test_code_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'test-code' command parser."""
    # Get dynamic examples
    example_code = get_example_test_code()
    combine_example = get_combine_classes_example()
    
    description = (
        'Test code snippets using the library with automatic imports.\n\n'
        'This command:\n'
        '- Automatically generates all necessary import statements\n'
        '- Executes your code in a safe environment\n'
        '- Shows output or error messages\n'
        '- Helps validate your understanding before implementation\n\n'
        'Code can be provided directly or loaded from a file.'
    )
    
    epilog = 'Examples:\n'
    epilog += f'  cjm-tailwind-explore test-code "{example_code}"\n'
    epilog += '  cjm-tailwind-explore test-code --file snippet.py\n'
    epilog += f'  cjm-tailwind-explore test-code "{combine_example}" --show-imports'
    
    parser = subparsers.add_parser(
        'test-code', 
        help='Test code snippets using the library',
        description=description,
        epilog=epilog,
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    parser.add_argument(
        'code',
        type=str,
        nargs='?',
        help='Code to test (or use --file to load from file)'
    )
    parser.add_argument(
        '--file', '-f',
        type=str,
        help='Load code from a file'
    )
    parser.add_argument(
        '--show-imports',
        action='store_true',
        help='Show the generated imports'
    )
    parser.add_argument(
        '--timeout',
        type=int,
        default=10,
        help='Execution timeout in seconds (default: 10)'
    )
    return parser

In [ ]:
#| export
def add_core_utils_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'core-utils' command parser."""
    return subparsers.add_parser('core-utils', help='List core utility functions')

In [ ]:
#| export
def add_core_util_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'core-util' command parser."""
    parser = subparsers.add_parser('core-util', help='Show source code for a core utility')
    
    # Get examples dynamically
    example_utils = get_example_core_utils()
    
    parser.add_argument(
        'name',
        type=str,
        help=f'Utility name (e.g., {example_utils})' if example_utils else 'Utility name'
    )
    return parser

In [ ]:
#| export
def add_imports_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'imports' command parser."""
    # Get dynamic examples
    example_modules = get_example_modules(limit=2)
    
    description = (
        'Generate recommended import statements for using the library.\n\n'
        'This command generates:\n'
        '- FastHTML common imports (Div, P, Button, etc.)\n'
        '- Core utility imports (combine_classes, get_tailwind_headers)\n'
        '- Factory imports from selected modules\n'
        '- Helper function imports from selected modules\n\n'
        'By default, includes imports from all available modules.\n'
        'Use --modules to limit imports to specific modules only.'
    )
    
    if example_modules:
        description += f'\n\nExample: --modules {example_modules}'
    
    parser = subparsers.add_parser(
        'imports', 
        help='Show recommended import statements',
        description=description
    )
    parser.add_argument(
        '--modules', '-m',
        type=str,
        nargs='+',
        help='Specific modules to include (default: all modules)'
    )
    return parser

## Command Dispatcher

Function to handle command execution:

In [ ]:
#| export
def add_scan_parser(
    subparsers  # TODO: Add type hint and description
): # TODO: Add type hint
    """Add the 'scan' command parser."""
    description = (
        'Scan Python code for replaceable CSS class patterns.\n\n'
        'This command analyzes Python code to find hardcoded CSS classes\n'
        'that could be replaced with library utilities.\n\n'
        'Features:\n'
        '- Identifies cls= patterns in Python code\n'
        '- Extracts CSS classes from strings and combine_classes calls\n'
        '- Matches against available library utilities\n'
        '- Provides migration suggestions with CLI commands\n\n'
        'Supported inputs:\n'
        '- Direct code strings\n'
        '- Python files (.py)\n'
        '- Jupyter notebooks (.ipynb)\n\n'
        'The tool will auto-detect the input type, or you can specify it explicitly.'
    )
    
    epilog = '''Examples:
  # Scan a code snippet
  cjm-tailwind-explore scan 'Div("Hello", cls="flex items-center px-4")'
  
  # Scan a Python file
  cjm-tailwind-explore scan app.py
  cjm-tailwind-explore scan --file components/navbar.py
  
  # Scan a Jupyter notebook
  cjm-tailwind-explore scan notebook.ipynb
  
  # Specify input type explicitly
  cjm-tailwind-explore scan --type python_file app.py
  cjm-tailwind-explore scan --type notebook example.ipynb
  
  # Only show analysis without suggestions
  cjm-tailwind-explore scan app.py --no-suggestions'''
    
    parser = subparsers.add_parser(
        'scan', 
        help='Scan code for replaceable CSS patterns',
        description=description,
        epilog=epilog,
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    
    parser.add_argument(
        'input',
        type=str,
        help='Code string, Python file path, or Jupyter notebook path'
    )
    
    parser.add_argument(
        '--file', '-f',
        action='store_true',
        help='Treat input as a file path (auto-detected by default)'
    )
    
    parser.add_argument(
        '--type', '-t',
        type=str,
        choices=['code', 'python_file', 'notebook'],
        help='Explicitly specify input type (default: auto-detect)'
    )
    
    parser.add_argument(
        '--no-suggestions',
        action='store_true',
        help='Only show analysis without migration suggestions'
    )
    
    return parser

In [ ]:
#| export
def dispatch_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Dispatch the parsed arguments to the appropriate handler."""
    if args.command == 'modules':
        display_modules()
    
    elif args.command == 'factories':
        if args.module:
            display_module_factories(args.module)
        else:
            display_all_factories()
    
    elif args.command == 'factory':
        display_factory_info(args.module, args.name)
    
    elif args.command == 'examples':
        if args.module:
            display_module_examples(args.module)
        else:
            display_all_examples()
    
    elif args.command == 'example':
        display_example_source(args.module, args.feature)
    
    elif args.command == 'helpers':
        if args.module:
            display_module_helpers(args.module)
        else:
            display_all_helpers()
    
    elif args.command == 'helper':
        display_helper_source(args.module, args.name)
    
    elif args.command == 'search':
        handle_search_command(args)
    
    elif args.command == 'test-code':
        handle_test_code_command(args)
    
    elif args.command == 'core-utils':
        display_core_utilities()
    
    elif args.command == 'core-util':
        display_core_utility_source(args.name)
    
    elif args.command == 'imports':
        display_imports(args.modules)
    
    elif args.command == 'scan':
        handle_scan_command(args)

In [ ]:
#| export
def handle_search_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Handle the search command."""
    # Determine which content types to search
    if args.search_in == 'all':
        content_types = None  # Will search all types
    else:
        content_types = [args.search_in]
    
    # Perform search
    results = search_all(
        args.query,
        content_types=content_types,
        include_source=args.include_source,
        case_sensitive=args.case_sensitive
    )
    
    # Display results
    display_search_results(results, args.query)

In [ ]:
#| export
def handle_test_code_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Handle the test-code command."""
    # Get code from command line or file
    code = None
    if args.file:
        code = load_code_from_file(args.file)
        if code is None:
            return  # Error already printed
    elif args.code:
        code = args.code
    else:
        print("Error: You must provide code to test either as an argument or via --file")
        print("\nExamples:")
        print('  cjm-tailwind-explore test-code "print(str(p(4)))"')
        print('  cjm-tailwind-explore test-code --file test_snippet.py')
        return
    
    # Execute the code
    success, stdout, stderr = execute_test_code(
        code,
        show_imports=args.show_imports,
        timeout=args.timeout
    )
    
    # Display results
    display_test_code_result(success, stdout, stderr, code)

## Main Parser Setup

Function to set up the main argument parser:

In [ ]:
#| export
def handle_scan_command(
    args  # TODO: Add type hint and description
): # TODO: Add type hint
    """Handle the scan command."""
    # Determine input type
    input_type = None
    if args.type:
        # Map string to InputType enum
        type_map = {
            'code': InputType.CODE,
            'python_file': InputType.PYTHON_FILE,
            'notebook': InputType.NOTEBOOK
        }
        input_type = type_map.get(args.type)
    elif args.file:
        # Force file interpretation
        input_type = None  # Let auto-detect handle it
    
    # Display analysis with or without suggestions
    if args.no_suggestions:
        display_input_analysis(args.input, input_type)
    else:
        analyze_and_suggest_input(args.input, input_type)

In [ ]:
#| export
def setup_argument_parser(
): # TODO: Add type hint
    """Set up the main argument parser with all subcommands."""
    
    # Get dynamic examples
    example_modules = get_example_modules(limit=3)
    example_factories = get_example_factories(limit=4)
    first_module = get_example_modules(limit=1)
    first_factory = get_example_factories(limit=1)
    example_feature = get_example_features(limit=1)
    example_test_code = get_example_test_code()
    
    # Create comprehensive description
    description = f"""
cjm-fasthtml-tailwind CLI Explorer

This tool helps you explore the cjm-fasthtml-tailwind library, which provides:
- Python-native Tailwind CSS v4 utility class builders for FastHTML projects
- Type-safe, dynamic CSS class generation without hardcoded strings
- Comprehensive utility factories ({example_factories}, etc.)
- Helper functions for common patterns
- Full integration with FastHTML components

Purpose: This CLI tool enables autonomous exploration of the library's API by:
- Discovering all available utility modules and their documentation
- Listing factory instances with their built-in documentation
- Showing usage examples from test functions
- Providing source code for helper functions
- Searching across all library components
- Testing code snippets with automatic imports
- Generating recommended import statements
- Scanning existing code for replaceable CSS patterns

All information is dynamically extracted from the library itself - nothing is hardcoded.
""".strip()
    
    # Create epilog with guidance
    epilog = f"""
Getting Started:
  1. List all modules:     cjm-tailwind-explore modules
  2. View factories:       cjm-tailwind-explore factories
  3. Search for patterns:  cjm-tailwind-explore search <query>
  4. Test code:           cjm-tailwind-explore test-code "<code>"
  5. Get imports:         cjm-tailwind-explore imports
  6. Scan existing code:  cjm-tailwind-explore scan <file>

Exploration Workflow:
  - Start with 'modules' to see available utility categories
  - Use 'factories -m <module>' to explore specific modules
  - Use 'factory <module> <name>' for detailed factory information
  - Use 'examples' to see test-based usage patterns
  - Use 'search' to find specific functionality
  - Use 'test-code' to verify your understanding
  - Use 'scan' to analyze existing code for migration opportunities

Key Concepts:
  - Factories: Objects that generate CSS classes (e.g., {example_factories})
  - Modules: Categories of utilities ({example_modules}, etc.)
  - Examples: Test functions demonstrating usage patterns
  - Helpers: Convenience functions for common patterns

Tips for Coding Assistants:
  - Use 'search --include-source' to find usage patterns in code
  - Use 'test-code' to validate generated code before using it
  - Use 'imports' to get all necessary import statements
  - Use 'scan' to identify replaceable hardcoded CSS classes
  - Factory names are intuitive: {example_factories}
  - Combine utilities with combine_classes() function
  - All factories support method chaining and attribute access

Example Usage Flow:
  cjm-tailwind-explore modules                    # See what's available
  cjm-tailwind-explore factories -m {first_module}       # Explore {first_module} utilities
  cjm-tailwind-explore factory {first_module} {first_factory}          # Learn about {first_factory} factory
  cjm-tailwind-explore example {first_module} {example_feature}      # See usage examples
  cjm-tailwind-explore test-code '{example_test_code}'   # Test your understanding
  cjm-tailwind-explore scan app.py                # Analyze existing code
""".strip()
    
    parser = argparse.ArgumentParser(
        description=description,
        epilog=epilog,
        formatter_class=argparse.RawDescriptionHelpFormatter,
        prog="cjm-tailwind-explore"
    )
    
    # Create subcommands
    subparsers = parser.add_subparsers(dest='command', help='Available commands')
    
    # Add all subcommand parsers
    add_modules_parser(subparsers)
    add_factories_parser(subparsers)
    add_factory_parser(subparsers)
    add_examples_parser(subparsers)
    add_example_parser(subparsers)
    add_helpers_parser(subparsers)
    add_helper_parser(subparsers)
    add_search_parser(subparsers)
    add_test_code_parser(subparsers)
    add_core_utils_parser(subparsers)
    add_core_util_parser(subparsers)
    add_imports_parser(subparsers)
    add_scan_parser(subparsers)  # Add the scan parser
    
    return parser

## CLI Entry Point

Main function for the console script:

In [ ]:
#| export
import argparse

def main(
): # TODO: Add type hint
    """CLI entry point for exploring cjm-fasthtml-tailwind utilities."""
    # Set up the argument parser
    parser = setup_argument_parser()
    
    # Parse arguments
    args = parser.parse_args()
    
    # If no command specified, show help
    if args.command is None:
        parser.print_help()
        return
    
    # Dispatch to the appropriate command handler
    dispatch_command(args)

## CLI Usage

Once installed, you can use the CLI tool from the command line:

```bash
# Show help
cjm-tailwind-explore --help

# List all utility modules
cjm-tailwind-explore modules

# List all factories across all modules
cjm-tailwind-explore factories

# List factories for a specific module
cjm-tailwind-explore factories --module spacing
cjm-tailwind-explore factories -m sizing

# Show detailed info for a specific factory
cjm-tailwind-explore factory spacing p
cjm-tailwind-explore factory spacing m
cjm-tailwind-explore factory sizing w
cjm-tailwind-explore factory sizing h

# Show all usage examples across all modules
cjm-tailwind-explore examples

# Show usage examples for a specific module
cjm-tailwind-explore examples --module spacing
cjm-tailwind-explore examples -m sizing

# Show source code for a specific example
cjm-tailwind-explore example spacing basic
cjm-tailwind-explore example spacing directional
cjm-tailwind-explore example sizing width

# Show all helper functions across all modules
cjm-tailwind-explore helpers

# Show helper functions for a specific module
cjm-tailwind-explore helpers --module spacing
cjm-tailwind-explore helpers -m sizing

# Show source code for a specific helper function
cjm-tailwind-explore helper spacing pad
cjm-tailwind-explore helper spacing margin

# List core utility functions
cjm-tailwind-explore core-utils

# Show source code and usage for a core utility
cjm-tailwind-explore core-util combine_classes
cjm-tailwind-explore core-util get_tailwind_headers

# Show recommended import statements
cjm-tailwind-explore imports
cjm-tailwind-explore imports --modules spacing sizing
cjm-tailwind-explore imports -m flexbox_and_grid layout

# Search across all library components
cjm-tailwind-explore search padding
cjm-tailwind-explore search "margin"
cjm-tailwind-explore search negative

# Search only in specific content types
cjm-tailwind-explore search width --in factories
cjm-tailwind-explore search negative --in examples
cjm-tailwind-explore search pad --in helpers
cjm-tailwind-explore search spacing --in modules

# Search including source code
cjm-tailwind-explore search "p(" --include-source
cjm-tailwind-explore search "m.auto" --include-source
cjm-tailwind-explore search "combine_classes" --include-source

# Case-sensitive search
cjm-tailwind-explore search "BaseFactory" --case-sensitive

# Test code snippets
cjm-tailwind-explore test-code "print(str(p(4)))"
cjm-tailwind-explore test-code "print(str(w.full))"
cjm-tailwind-explore test-code "print(combine_classes(p(4), m.auto, 'flex'))"

# Test code from a file
cjm-tailwind-explore test-code --file test_snippet.py

# Test code with generated imports shown
cjm-tailwind-explore test-code "print(str(p(4)))" --show-imports

# Test code with custom timeout
cjm-tailwind-explore test-code --file complex_test.py --timeout 30

# Scan code for replaceable CSS patterns
cjm-tailwind-explore scan 'Div("Hello", cls="flex items-center px-4")'
cjm-tailwind-explore scan app.py
cjm-tailwind-explore scan notebook.ipynb
cjm-tailwind-explore scan --file components/navbar.py
cjm-tailwind-explore scan app.py --no-suggestions
cjm-tailwind-explore scan --type notebook analysis.ipynb
```

### Pattern Scanning

The `scan` command analyzes Python code to find hardcoded CSS classes that could be replaced with library utilities:

```bash
# Scan a code snippet
cjm-tailwind-explore scan 'Button("Click", cls="px-4 py-2 bg-blue-500")'

# Scan a Python file
cjm-tailwind-explore scan app.py
cjm-tailwind-explore scan --file src/components.py

# Scan a Jupyter notebook
cjm-tailwind-explore scan notebook.ipynb

# Only show analysis without migration suggestions
cjm-tailwind-explore scan app.py --no-suggestions

# Explicitly specify input type
cjm-tailwind-explore scan --type python_file app.py
cjm-tailwind-explore scan --type notebook demo.ipynb
```

Example output:
```
Pattern Analysis Report
============================================================
Input Type: python_file
File: app.py
Total cls= patterns found: 5
Unique CSS classes: 12

Replaceable Classes: 8/12
  - Exact matches: 6
  - Pattern matches: 2
  - No matches: 4

CSS Class Analysis Results:
============================================================

✓ Exact Matches (6):
  - flex
  - items-center
  - px-4
  - py-2
  ...

Migration Suggestions
============================================================

✓ flex:
  → View example: cjm-tailwind-explore example layout display

✓ px-4:
  → View example: cjm-tailwind-explore example spacing directional
  → View helper: cjm-tailwind-explore helper spacing pad
```

### Core Utilities

The `combine_classes` function is essential for combining utility classes:

```bash
# View documentation and source for combine_classes
cjm-tailwind-explore core-util combine_classes

# Search for usage examples
cjm-tailwind-explore search combine_classes --include-source

# Test it directly
cjm-tailwind-explore test-code "
result = combine_classes(
    p(4),
    m.auto,
    w.full,
    'bg-blue-500',
    None,  # None values are filtered out
    'text-white'
)
print('Combined:', result)
"
```

### Getting Started with Imports

```bash
# Get all recommended imports
cjm-tailwind-explore imports

# Get imports for specific modules only
cjm-tailwind-explore imports --modules spacing sizing

# Use the imports in a new file
cjm-tailwind-explore imports > my_imports.py
```

### Test Code Examples

The `test-code` command allows you to quickly test code snippets before implementing them:

```bash
# Test basic factory usage
cjm-tailwind-explore test-code "
# Test padding utilities
print('p(4):', str(p(4)))
print('p.x(8):', str(p.x(8)))
print('p.y.auto:', str(p.y.auto))
"

# Test combining classes
cjm-tailwind-explore test-code "
card_classes = combine_classes(
    p(6),
    m(4),
    'rounded-lg',
    'shadow-md'
)
print('Card classes:', card_classes)
"

# Test helper functions
cjm-tailwind-explore test-code "
# Using the pad helper
padding = pad(x=4, y=2)
print('Padding classes:', padding)

# Using the margin helper
spacing = margin(t=4, b=8, x='auto')
print('Margin classes:', spacing)
"

# Test a complete component
cjm-tailwind-explore test-code "
# Create a button component
button = Button(
    'Click me',
    cls=combine_classes(
        p.x(4),
        p.y(2),
        'bg-blue-500',
        'text-white',
        'rounded-md',
        'hover:bg-blue-600'
    )
)
print('Button HTML:', button)
"
```

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()